# Ejemplo de DNN con Data Augmentation

Por cierto, no funciona en este caso, aunque no he hecho el esfuerzo de optimizar los hiperparámetros.

Importamos las librerias y hacemos el cuaderno repetible fijando la semilla de los generadores aleatorios.

In [1]:
# Import TensorFlow and Keras (Keras3 is integrated into TensorFlow)
import keras
from keras import layers
import tensorflow as tf

tf.keras.utils.set_random_seed(42)  # Puedes usar cualquier número como semilla




### Cargar y pre-procesar los datos

Vamos a utilizar el dataset fashion mnist, con imágenes en escala de grises de prendas de vestir extraidas de zalando.

| Label | Description |
|---|---|
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

In [2]:
# Cargamos Fashion MNIST data
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Add channel dimension (the images are grayscale)
x_train = x_train[..., None]  # shape becomes (60000, 28, 28, 1)
x_test = x_test[..., None]

# Normalizamos en el rango [0, 1]
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0


### Primero sin Data Augmentation

Construimos el modelo con tres capas densas

In [3]:
# Build a simple model with only Dense layers
modelNDA = keras.Sequential(
    [
        keras.Input(shape=(28, 28,1)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(10, activation="softmax"),  # 10 clases
    ]
)

# Resumen del modelo
modelNDA.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 109,386 (427.29 KB)

 Trainable params: 109,386 (427.29 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Compilar
modelNDA.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

# Entrenar
historyNDA = modelNDA.fit(
    x_train, y_train,
    batch_size=64,
    epochs=5,
    validation_data=(x_test, y_test)

)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7710 - loss: 0.6713 - val_accuracy: 0.8445 - val_loss: 0.4316
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8609 - loss: 0.3911 - val_accuracy: 0.8563 - val_loss: 0.3896
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8744 - loss: 0.3467 - val_accuracy: 0.8606 - val_loss: 0.3729
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8816 - loss: 0.3191 - val_accuracy: 0.8694 - val_loss: 0.3618
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8889 - loss: 0.2968 - val_accuracy: 0.8713 - val_loss: 0.3580


In [5]:
# Evaluar con los datos de test
test_loss, test_acc = modelNDA.evaluate(x_test, y_test, verbose=2)
print("Test accuracy:", test_acc)


313/313 - 1s - 3ms/step - accuracy: 0.8713 - loss: 0.3580
Test accuracy: 0.8712999820709229


### Definir el Data Augmentation Pipeline

creamos un proceso sencillo de data augmentation utilizando las capas de preproceso de keras. Aleatoriamente tota, amplia y voltea las imagenes.

In [10]:
# Data augmentation pipeline
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
    ]
)


### Construir el modelo


In [11]:
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28,1)),
        # Aplicar Data Augmentation durante el training
        data_augmentation,
        # Flatten
        layers.Flatten(),
        # Dense layers
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

# model summary
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_3 (Sequential)            │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 109,386 (427.29 KB)

 Trainable params: 109,386 (427.29 KB)

 Non-trainable params: 0 (0.00 B)

### Compilar y entrenar


In [14]:
# Compilar

import numpy as np


model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

# Duplicamos los casos, ya que el aumentador nos los va a variar
x_train_big = np.repeat(x_train, 4, axis=0)
y_train_big = np.repeat(y_train, 4, axis=0)

history = model.fit(
    x_train_big, y_train_big,
    batch_size=64,
    epochs=5,
    validation_data=(x_test, y_test)

)


Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.8257 - loss: 0.4762 - val_accuracy: 0.8221 - val_loss: 0.4809
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.8276 - loss: 0.4711 - val_accuracy: 0.8192 - val_loss: 0.4850
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.8285 - loss: 0.4686 - val_accuracy: 0.8312 - val_loss: 0.4604
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.8284 - loss: 0.4689 - val_accuracy: 0.8315 - val_loss: 0.4601
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - accuracy: 0.8313 - loss: 0.4631 - val_accuracy: 0.8219 - val_loss: 0.4897


### Evaluar


In [15]:

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("Test accuracy:", test_acc)


313/313 - 1s - 2ms/step - accuracy: 0.8219 - loss: 0.4897
Test accuracy: 0.8219000101089478
